In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from pathlib import Path
import requests
from my_tokenizer import CharDataset
from my_gpt import SmolGPT

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


#### Unit testing

In [12]:
!pytest my_tests.py -v

============================= test session starts =============================
platform win32 -- Python 3.11.5, pytest-9.0.1, pluggy-1.6.0 -- C:\Git\learning_pytorch\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Git\mini_project_language_model\src
plugins: anyio-4.10.0
collecting ... collected 6 items

my_tests.py::test_tokenizer_roundtrip PASSED                             [ 16%]
my_tests.py::test_single_attention_head PASSED                           [ 33%]
my_tests.py::test_multi_attention_head PASSED                            [ 50%]
my_tests.py::test_ffn PASSED                                             [ 66%]
my_tests.py::test_transformer_block PASSED                               [ 83%]
my_tests.py::test_full_model PASSED                                      [100%]

============================== 6 passed in 2.05s ==============================


#### Get Data using DataSet / DataLoader

In [13]:
from torch.utils.data import DataLoader

# Load Shakespeare data
with open('../data/shakespeare.txt', 'r') as f:
    text = f.read()

# Build vocab from entire text ONCE
vocab = sorted(list(set(text)))

# Split into train/val (80/20)
n = int(0.8 * len(text))
train_text, val_text = text[:n], text[n:]

# Create datasets with shared vocab
train_dataset = CharDataset(train_text, block_size=128, vocab=vocab)
val_dataset = CharDataset(val_text, block_size=128, vocab=vocab)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

vocab_size = train_dataset.get_vocab_size()

In [14]:
@torch.no_grad()
def estimate_loss(model, train_loader, val_loader, device, eval_batches=50):
    """
    Estimate loss on train and val sets
    Args:
        model: GPT model
        train_loader: training DataLoader
        val_loader: val DataLoader
        device: cpu or cuda
        eval_batches: nb of batches to average over
    Returns:
        a Dictionary with 'train' and 'val' losses
    """
    out = {}
    model.eval()
    
    for split, loader in [('train', train_loader), ('val', val_loader)]:
        losses = []
        for i, (x, y) in enumerate(loader):
            if i >= eval_batches:
                break
            x, y = x.to(device), y.to(device)
            _, loss = model(x, y)
            losses.append(loss.item())
        out[split] = sum(losses) / len(losses)
    
    model.train()
    return out

#### Setting hyperparameters

In [21]:
# The project was tested with 12 layers, 8 attention heads, and 768 embedding dimensions, on a single GPU.

## big
# n_embd=768
# block_size=128
# batch_size = 64
# num_head=8
# num_layers=12
# dropout=0.1
# learning_rate = 1e-4

## small
n_embd = 384
batch_size = 32
block_size = 128
num_head = 8
num_layers = 6
dropout = 0.3
learning_rate = 1e-4


#### Training loop

In [22]:
# Create model
model = SmolGPT(
    vocab_size=vocab_size,
    n_embd=n_embd,
    block_size=block_size,
    num_head=num_head,
    num_layers=num_layers,
    dropout=dropout
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

training_steps = 6000
eval_interval = training_steps // 100
iter = 0

# add early stopping
best_val_loss = float('inf')
patience = 5  # stop if val loss doesn't improve for 5 evals
patience_counter = 0

for x, y in train_loader:
    if iter % eval_interval == 0:
        losses = estimate_loss(model, train_loader, val_loader, device)
        print(f"Step {iter:4d} | Train: {losses['train']:.4f} | Val: {losses['val']:.4f}")
        
        # Early stopping check
        if losses['val'] < best_val_loss:
            best_val_loss = losses['val']
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"  (no improvement, {patience_counter}/{patience})")
            
            if patience_counter >= patience:
                print(f"Early stopping at step {iter}")
                break
        
    x, y = x.to(device), y.to(device)

    logits, loss = model(x, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    iter += 1
    if iter >= training_steps:
        break

print("Training complete!")


Step    0 | Train: 4.3502 | Val: 4.3381
Step   60 | Train: 2.6369 | Val: 2.6093
Step   60 | Train: 2.6369 | Val: 2.6093
Step  120 | Train: 2.5066 | Val: 2.4941
Step  120 | Train: 2.5066 | Val: 2.4941
Step  180 | Train: 2.4697 | Val: 2.4628
Step  180 | Train: 2.4697 | Val: 2.4628
Step  240 | Train: 2.4349 | Val: 2.4324
Step  240 | Train: 2.4349 | Val: 2.4324
Step  300 | Train: 2.4030 | Val: 2.4022
Step  300 | Train: 2.4030 | Val: 2.4022
Step  360 | Train: 2.3599 | Val: 2.3704
Step  360 | Train: 2.3599 | Val: 2.3704
Step  420 | Train: 2.3170 | Val: 2.3204
Step  420 | Train: 2.3170 | Val: 2.3204
Step  480 | Train: 2.2605 | Val: 2.2642
Step  480 | Train: 2.2605 | Val: 2.2642
Step  540 | Train: 2.1995 | Val: 2.2136
Step  540 | Train: 2.1995 | Val: 2.2136
Step  600 | Train: 2.1435 | Val: 2.1618
Step  600 | Train: 2.1435 | Val: 2.1618
Step  660 | Train: 2.0908 | Val: 2.1164
Step  660 | Train: 2.0908 | Val: 2.1164
Step  720 | Train: 2.0525 | Val: 2.0670
Step  720 | Train: 2.0525 | Val: 2.0670


#### Save model

In [ ]:
model_path = Path('../models/mini_transformer_model_simple_small.pth')
torch.save(model.state_dict(), model_path)

#### Generation

In [23]:
model.eval()
dataset = CharDataset(text, block_size=128)
max_new_tokens = 500
with torch.no_grad():
    context = "O God, O God!"
    tokens = dataset.encode(context)
    #not sure about this, wanting to change the tensor shape
    idx = torch.tensor(tokens).view(1, len(tokens)).to(device)
    y = model.generate(idx, max_new_tokens)
    completion = dataset.decode(y[0].tolist())
    print(completion)

O God, O God!
What is the preparer, unhath the fast, be Edward,
See and stander these by point life
Erst by Gloucester's throne; new, and, for them see,
Renses that that your Richard's eyes, and alone.

MENENIUS:
Ay, come, cut my sen.

KING EDWARD IV:
He shall offended with Clarence?

SICINIUS:
'Tis rue:

MONTAGUE:
A great; my lord, sir,
Who enspect swifts their to title in son.

KING RICHARD III:
Let us queen him at a sun his charise,
Since at tell us us let me at the sompician,
If you will him, for so that
